<a href="https://colab.research.google.com/github/Yatrogenesis/Fooocus/blob/main/FOOOCUS_ALL_SYLES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Ejecución automática de Fooocus en Colab

# Verificar GPU
def verificar_gpu():
    import torch
    if not torch.cuda.is_available():
        raise RuntimeError("No se detectó una GPU disponible. Configura el entorno con GPU: Entorno de ejecución > Cambiar tipo de entorno > GPU")

# Actualizar e instalar dependencias del sistema
!apt-get update -qq
!apt-get install -y git git-lfs

# Instalar Torch compatible con Colab
!pip install torch torchvision --index-url https://download.pytorch.org/whl/cu121

# Verificar GPU disponible
verificar_gpu()

# Clonar e instalar Fooocus
!git clone https://github.com/lllyasviel/Fooocus.git
%cd Fooocus
!git lfs install
!git lfs pull

# Instalar dependencias específicas de Fooocus
!pip install -r requirements_versions.txt

# Lanzar Fooocus y compartir enlace
!python launch.py --share

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.12).
git-lfs is already the newest version (3.0.2-1ubuntu0.3).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.
Looking in indexes: https://download.pytorch.org/whl/cu121
INFO: pip is looking at multiple versions of torch to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.5/780.5 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 80.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 54.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 113.3 MB/s eta 0:00:00
     

[System ARGV] ['launch.py', '--share']
Python 3.11.11 (main, Dec  4 2024, 08:55:07) [GCC 11.4.0]
Fooocus version: 2.5.5
[Cleanup] Attempting to delete content of temp dir /tmp/fooocus
[Cleanup] Cleanup successful
Downloading: "https://huggingface.co/lllyasviel/misc/resolve/main/xlvaeapp.pth" to /content/Fooocus/models/vae_approx/xlvaeapp.pth

100% 209k/209k [00:00<00:00, 6.24MB/s]
Downloading: "https://huggingface.co/lllyasviel/misc/resolve/main/vaeapp_sd15.pt" to /content/Fooocus/models/vae_approx/vaeapp_sd15.pth

100% 209k/209k [00:00<00:00, 5.93MB/s]
Downloading: "https://huggingface.co/mashb1t/misc/resolve/main/xl-to-v1_interposer-v4.0.safetensors" to /content/Fooocus/models/vae_approx/xl-to-v1_interposer-v4.0.safetensors

100% 5.40M/5.40M [00:00<00:00, 65.9MB/s]
Downloading: "https://huggingface.co/lllyasviel/misc/resolve/main/fooocus_expansion.bin" to /content/Fooocus/models/prompt_expansion/fooocus_expansion/pytorch_model.bin

100% 335M/335M [00:01<00:00, 328MB/s]
Downloading: "

In [ ]:
# CFD Simulator para Fluidos en Rotación - Versión Google Colab
# Autor: Claude AI
# Descripción: Simulación CFD de fluidos en movimiento giratorio dentro de geometrías variables

# Celda 1: Instalación de paquetes y configuración inicial
# Ejecutar esta celda primero
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
from scipy.integrate import odeint
import time
from IPython.display import clear_output

# Configuración de visualización para Colab
%matplotlib inline
plt.rcParams['figure.figsize'] = [12, 8]

print("Configuración completada. Paquetes importados correctamente.")

# Celda 2: Definición de la clase FluidSimulator
class FluidSimulator:
    """
    Simulador CFD para fluidos en movimiento giratorio dentro de geometrías variables
    que pueden transformarse desde esferas a toroides, con un eje central conductor.
    """

    def __init__(self, grid_size=30, sphere_radius=1.0, time_step=0.01, viscosity=0.1):
        """
        Inicializa el simulador con parámetros básicos

        Args:
            grid_size: Número de puntos en cada dimensión
            sphere_radius: Radio de la esfera límite externa
            time_step: Paso de tiempo para la simulación
            viscosity: Viscosidad cinemática del fluido
        """
        self.grid_size = grid_size
        self.sphere_radius = sphere_radius
        self.time_step = time_step
        self.viscosity = viscosity

        # Crear mallas para coordenadas
        self.x = np.linspace(-sphere_radius, sphere_radius, grid_size)
        self.y = np.linspace(-sphere_radius, sphere_radius, grid_size)
        self.z = np.linspace(-sphere_radius, sphere_radius, grid_size)
        self.X, self.Y, self.Z = np.meshgrid(self.x, self.y, self.z, indexing='ij')

        # Campos iniciales de velocidad (U, V, W) y presión (P)
        self.U = np.zeros((grid_size, grid_size, grid_size))
        self.V = np.zeros((grid_size, grid_size, grid_size))
        self.W = np.zeros((grid_size, grid_size, grid_size))
        self.P = np.zeros((grid_size, grid_size, grid_size))

        # Definir geometría inicial (cilíndrica con base plana)
        self.container_type = "cylinder"
        self.inner_radius = 0.4 * sphere_radius
        self.outer_radius = 0.8 * sphere_radius
        self.cylinder_height = 1.6 * sphere_radius
        self.toroid_factor = 0.0  # Factor para transformación a toroide (0-1)

        # Propiedades del eje central
        self.central_axis_radius = 0.05 * sphere_radius
        self.central_axis_rotation = 0.0  # Velocidad angular del eje (rad/s)

        # Crear máscara inicial para determinar qué celdas están dentro del dominio
        self.update_geometry_mask()

        # Estado del sistema
        self.time = 0.0
        self.iteration = 0

    def update_geometry_mask(self):
        """Actualiza la máscara de geometría según la configuración actual"""
        # Coordenadas radiales y altura
        R = np.sqrt(self.X**2 + self.Y**2)

        # Máscara para la esfera externa límite
        sphere_mask = self.X**2 + self.Y**2 + self.Z**2 <= self.sphere_radius**2

        # Máscara para el eje central
        central_axis_mask = R <= self.central_axis_radius

        # Inicialmente todas las celdas están fuera del dominio
        self.mask = np.zeros((self.grid_size, self.grid_size, self.grid_size), dtype=bool)

        if self.container_type == "cylinder":
            # Cilindro: R <= radio_cilindro y -altura/2 <= Z <= altura/2
            cylinder_mask = (R <= self.inner_radius) & (np.abs(self.Z) <= self.cylinder_height/2)
            self.mask = cylinder_mask & sphere_mask & ~central_axis_mask

        elif self.container_type == "sphere":
            # Esfera interior
            inner_sphere_mask = self.X**2 + self.Y**2 + self.Z**2 <= self.inner_radius**2
            self.mask = inner_sphere_mask & ~central_axis_mask

        elif self.container_type == "toroid":
            # Radio mayor del toroide
            major_radius = (self.inner_radius + self.outer_radius) / 2
            # Radio menor del toroide
            minor_radius = (self.outer_radius - self.inner_radius) / 2

            # Distancia desde el punto al círculo central del toroide en el plano XY
            dist_from_circle = np.abs(np.sqrt(self.X**2 + self.Y**2) - major_radius)

            # Máscara del toroide
            toroid_mask = dist_from_circle**2 + self.Z**2 <= minor_radius**2
            self.mask = toroid_mask & sphere_mask & ~central_axis_mask

        elif self.container_type == "hybrid":
            # Interpolación entre cilindro y toroide
            R_major = (self.inner_radius + self.outer_radius) / 2
            R_minor = (self.outer_radius - self.inner_radius) / 2

            # Factor de ponderación para la transformación
            t = self.toroid_factor

            # Para t=0: cilindro, para t=1: toroide
            if t <= 0:
                cylinder_mask = (R <= self.inner_radius) & (np.abs(self.Z) <= self.cylinder_height/2)
                self.mask = cylinder_mask & sphere_mask & ~central_axis_mask
            elif t >= 1:
                dist_from_circle = np.abs(np.sqrt(self.X**2 + self.Y**2) - R_major)
                toroid_mask = dist_from_circle**2 + self.Z**2 <= R_minor**2
                self.mask = toroid_mask & sphere_mask & ~central_axis_mask
            else:
                # Interpolación de la geometría
                cylinder_center_dist = np.maximum(
                    np.abs(R - self.inner_radius/2),
                    np.abs(self.Z) - self.cylinder_height/2
                )
                dist_from_circle = np.abs(np.sqrt(self.X**2 + self.Y**2) - R_major)
                toroid_dist = np.sqrt(dist_from_circle**2 + self.Z**2) - R_minor

                # Combinar con interpolación suave
                hybrid_dist = (1 - t) * cylinder_center_dist + t * toroid_dist
                hybrid_mask = hybrid_dist <= 0
                self.mask = hybrid_mask & sphere_mask & ~central_axis_mask

    def set_container_type(self, container_type, toroid_factor=None):
        """
        Establece el tipo de contenedor para la simulación

        Args:
            container_type: "cylinder", "sphere", "toroid", o "hybrid"
            toroid_factor: Factor de transformación para "hybrid" (0-1)
        """
        self.container_type = container_type
        if toroid_factor is not None:
            self.toroid_factor = max(0.0, min(1.0, toroid_factor))
        self.update_geometry_mask()

    def set_rotation_speed(self, speed):
        """Establece la velocidad de rotación del eje central"""
        self.central_axis_rotation = speed

    def initialize_fluid_state(self, init_mode="rest"):
        """
        Inicializa el estado del fluido

        Args:
            init_mode: Modo de inicialización ("rest", "rigid_rotation", "random")
        """
        # Reiniciar campos
        self.U = np.zeros((self.grid_size, self.grid_size, self.grid_size))
        self.V = np.zeros((self.grid_size, self.grid_size, self.grid_size))
        self.W = np.zeros((self.grid_size, self.grid_size, self.grid_size))
        self.P = np.zeros((self.grid_size, self.grid_size, self.grid_size))

        if init_mode == "rest":
            # Fluido en reposo, ya inicializado a cero
            pass

        elif init_mode == "rigid_rotation":
            # Inicializar con rotación rígida
            omega = 1.0  # Velocidad angular inicial
            for i in range(self.grid_size):
                for j in range(self.grid_size):
                    for k in range(self.grid_size):
                        if self.mask[i, j, k]:
                            # Componentes de velocidad tangencial
                            self.U[i, j, k] = -omega * self.Y[i, j, k]
                            self.V[i, j, k] = omega * self.X[i, j, k]

        elif init_mode == "random":
            # Inicializar con pequeñas perturbaciones aleatorias
            noise_amplitude = 0.05
            for i in range(self.grid_size):
                for j in range(self.grid_size):
                    for k in range(self.grid_size):
                        if self.mask[i, j, k]:
                            self.U[i, j, k] = noise_amplitude * (np.random.rand() - 0.5)
                            self.V[i, j, k] = noise_amplitude * (np.random.rand() - 0.5)
                            self.W[i, j, k] = noise_amplitude * (np.random.rand() - 0.5)

    def apply_boundary_conditions(self):
        """Aplica condiciones de frontera a los campos de velocidad"""
        # Condición de no deslizamiento en las paredes
        for i in range(self.grid_size):
            for j in range(self.grid_size):
                for k in range(self.grid_size):
                    if not self.mask[i, j, k]:
                        # Fuera del dominio del fluido, fijar velocidad a cero
                        self.U[i, j, k] = 0
                        self.V[i, j, k] = 0
                        self.W[i, j, k] = 0

        # Condición especial en el eje central (velocidad tangencial)
        R = np.sqrt(self.X**2 + self.Y**2)
        axis_mask = R <= self.central_axis_radius

        for i in range(self.grid_size):
            for j in range(self.grid_size):
                for k in range(self.grid_size):
                    if axis_mask[i, j, k] and (self.X[i,j,k]**2 + self.Y[i,j,k]**2 + self.Z[i,j,k]**2 <= self.sphere_radius**2):
                        # Aplicar rotación en el eje central
                        self.U[i, j, k] = -self.central_axis_rotation * self.Y[i, j, k]
                        self.V[i, j, k] = self.central_axis_rotation * self.X[i, j, k]
                        self.W[i, j, k] = 0

    def compute_velocity_gradient(self):
        """Calcula los gradientes de velocidad"""
        dx = self.x[1] - self.x[0]
        dy = self.y[1] - self.y[0]
        dz = self.z[1] - self.z[0]

        # Derivadas parciales usando diferencias finitas centradas
        dUdx = np.zeros_like(self.U)
        dUdy = np.zeros_like(self.U)
        dUdz = np.zeros_like(self.U)
        dVdx = np.zeros_like(self.V)
        dVdy = np.zeros_like(self.V)
        dVdz = np.zeros_like(self.V)
        dWdx = np.zeros_like(self.W)
        dWdy = np.zeros_like(self.W)
        dWdz = np.zeros_like(self.W)

        # Calcular gradientes en puntos interiores
        for i in range(1, self.grid_size - 1):
            for j in range(1, self.grid_size - 1):
                for k in range(1, self.grid_size - 1):
                    if self.mask[i, j, k]:
                        dUdx[i, j, k] = (self.U[i+1, j, k] - self.U[i-1, j, k]) / (2*dx)
                        dUdy[i, j, k] = (self.U[i, j+1, k] - self.U[i, j-1, k]) / (2*dy)
                        dUdz[i, j, k] = (self.U[i, j, k+1] - self.U[i, j, k-1]) / (2*dz)

                        dVdx[i, j, k] = (self.V[i+1, j, k] - self.V[i-1, j, k]) / (2*dx)
                        dVdy[i, j, k] = (self.V[i, j+1, k] - self.V[i, j-1, k]) / (2*dy)
                        dVdz[i, j, k] = (self.V[i, j, k+1] - self.V[i, j, k-1]) / (2*dz)

                        dWdx[i, j, k] = (self.W[i+1, j, k] - self.W[i-1, j, k]) / (2*dx)
                        dWdy[i, j, k] = (self.W[i, j+1, k] - self.W[i, j-1, k]) / (2*dy)
                        dWdz[i, j, k] = (self.W[i, j, k+1] - self.W[i, j, k-1]) / (2*dz)

        return dUdx, dUdy, dUdz, dVdx, dVdy, dVdz, dWdx, dWdy, dWdz

    def solve_pressure_poisson(self):
        """Resuelve la ecuación de Poisson para la presión usando Jacobi"""
        dx = self.x[1] - self.x[0]
        dy = self.y[1] - self.y[0]
        dz = self.z[1] - self.z[0]

        # Divergencia del campo de velocidad
        dUdx, dUdy, dUdz, dVdx, dVdy, dVdz, dWdx, dWdy, dWdz = self.compute_velocity_gradient()
        div = dUdx + dVdy + dWdz

        # Término fuente para la ecuación de Poisson
        rhs = div / self.time_step

        # Matriz inicial de presión
        p_new = np.copy(self.P)

        # Número de iteraciones para la solución de Poisson
        n_iter = 50

        # Resolver mediante el método iterativo de Jacobi
        for _ in range(n_iter):
            for i in range(1, self.grid_size - 1):
                for j in range(1, self.grid_size - 1):
                    for k in range(1, self.grid_size - 1):
                        if self.mask[i, j, k]:
                            p_new[i, j, k] = (1/6) * (
                                self.P[i+1, j, k] + self.P[i-1, j, k] +
                                self.P[i, j+1, k] + self.P[i, j-1, k] +
                                self.P[i, j, k+1] + self.P[i, j, k-1] -
                                dx**2 * rhs[i, j, k]
                            )

            # Actualizar campo de presión
            self.P = np.copy(p_new)

        return self.P

    def update_velocity_field(self):
        """Actualiza el campo de velocidad utilizando el esquema de proyección"""
        dx = self.x[1] - self.x[0]
        dy = self.y[1] - self.y[0]
        dz = self.z[1] - self.z[0]

        # Calcular gradientes de velocidad
        dUdx, dUdy, dUdz, dVdx, dVdy, dVdz, dWdx, dWdy, dWdz = self.compute_velocity_gradient()

        # Campos temporales para almacenar velocidades intermedias
        u_temp = np.copy(self.U)
        v_temp = np.copy(self.V)
        w_temp = np.copy(self.W)

        # Calcular campos intermedios (paso de advección-difusión)
        for i in range(1, self.grid_size - 1):
            for j in range(1, self.grid_size - 1):
                for k in range(1, self.grid_size - 1):
                    if self.mask[i, j, k]:
                        # Términos advectivos (esquema upwind)
                        if self.U[i, j, k] > 0:
                            du_dx = (self.U[i, j, k] - self.U[i-1, j, k]) / dx
                        else:
                            du_dx = (self.U[i+1, j, k] - self.U[i, j, k]) / dx

                        if self.V[i, j, k] > 0:
                            du_dy = (self.U[i, j, k] - self.U[i, j-1, k]) / dy
                        else:
                            du_dy = (self.U[i, j+1, k] - self.U[i, j, k]) / dy

                        if self.W[i, j, k] > 0:
                            du_dz = (self.U[i, j, k] - self.U[i, j, k-1]) / dz
                        else:
                            du_dz = (self.U[i, j, k+1] - self.U[i, j, k]) / dz

                        # Términos similares para V y W
                        if self.U[i, j, k] > 0:
                            dv_dx = (self.V[i, j, k] - self.V[i-1, j, k]) / dx
                        else:
                            dv_dx = (self.V[i+1, j, k] - self.V[i, j, k]) / dx

                        if self.V[i, j, k] > 0:
                            dv_dy = (self.V[i, j, k] - self.V[i, j-1, k]) / dy
                        else:
                            dv_dy = (self.V[i, j+1, k] - self.V[i, j, k]) / dy

                        if self.W[i, j, k] > 0:
                            dv_dz = (self.V[i, j, k] - self.V[i, j, k-1]) / dz
                        else:
                            dv_dz = (self.V[i, j, k+1] - self.V[i, j, k]) / dz

                        if self.U[i, j, k] > 0:
                            dw_dx = (self.W[i, j, k] - self.W[i-1, j, k]) / dx
                        else:
                            dw_dx = (self.W[i+1, j, k] - self.W[i, j, k]) / dx

                        if self.V[i, j, k] > 0:
                            dw_dy = (self.W[i, j, k] - self.W[i, j-1, k]) / dy
                        else:
                            dw_dy = (self.W[i, j+1, k] - self.W[i, j, k]) / dy

                        if self.W[i, j, k] > 0:
                            dw_dz = (self.W[i, j, k] - self.W[i, j, k-1]) / dz
                        else:
                            dw_dz = (self.W[i, j, k+1] - self.W[i, j, k]) / dz

                        # Términos difusivos (Laplaciano)
                        laplacian_u = (
                            self.U[i+1, j, k] - 2*self.U[i, j, k] + self.U[i-1, j, k]
                        ) / dx**2 + (
                            self.U[i, j+1, k] - 2*self.U[i, j, k] + self.U[i, j-1, k]
                        ) / dy**2 + (
                            self.U[i, j, k+1] - 2*self.U[i, j, k] + self.U[i, j, k-1]
                        ) / dz**2

                        laplacian_v = (
                            self.V[i+1, j, k] - 2*self.V[i, j, k] + self.V[i-1, j, k]
                        ) / dx**2 + (
                            self.V[i, j+1, k] - 2*self.V[i, j, k] + self.V[i, j-1, k]
                        ) / dy**2 + (
                            self.V[i, j, k+1] - 2*self.V[i, j, k] + self.V[i, j, k-1]
                        ) / dz**2

                        laplacian_w = (
                            self.W[i+1, j, k] - 2*self.W[i, j, k] + self.W[i-1, j, k]
                        ) / dx**2 + (
                            self.W[i, j+1, k] - 2*self.W[i, j, k] + self.W[i, j-1, k]
                        ) / dy**2 + (
                            self.W[i, j, k+1] - 2*self.W[i, j, k] + self.W[i, j, k-1]
                        ) / dz**2

                        # Actualizar velocidades intermedias (ecuación de Navier-Stokes sin término de presión)
                        u_temp[i, j, k] = self.U[i, j, k] - self.time_step * (
                            self.U[i, j, k] * du_dx +
                            self.V[i, j, k] * du_dy +
                            self.W[i, j, k] * du_dz
                        ) + self.time_step * self.viscosity * laplacian_u

                        v_temp[i, j, k] = self.V[i, j, k] - self.time_step * (
                            self.U[i, j, k] * dv_dx +
                            self.V[i, j, k] * dv_dy +
                            self.W[i, j, k] * dv_dz
                        ) + self.time_step * self.viscosity * laplacian_v

                        w_temp[i, j, k] = self.W[i, j, k] - self.time_step * (
                            self.U[i, j, k] * dw_dx +
                            self.V[i, j, k] * dw_dy +
                            self.W[i, j, k] * dw_dz
                        ) + self.time_step * self.viscosity * laplacian_w

        # Actualizar campos temporales
        self.U = u_temp
        self.V = v_temp
        self.W = w_temp

        # Resolver ecuación de Poisson para la presión
        self.solve_pressure_poisson()

        # Proyectar el campo de velocidad para hacerlo libre de divergencia
        for i in range(1, self.grid_size - 1):
            for j in range(1, self.grid_size - 1):
                for k in range(1, self.grid_size - 1):
                    if self.mask[i, j, k]:
                        # Gradiente de presión
                        dp_dx = (self.P[i+1, j, k] - self.P[i-1, j, k]) / (2*dx)
                        dp_dy = (self.P[i, j+1, k] - self.P[i, j-1, k]) / (2*dy)
                        dp_dz = (self.P[i, j, k+1] - self.P[i, j, k-1]) / (2*dz)

                        # Corrección del campo de velocidad
                        self.U[i, j, k] -= self.time_step * dp_

In [ ]:

# CFD Simulator para Fluidos en Rotación - Versión Google Colab
# Autor: Claude AI
# Descripción: Simulación CFD de fluidos en movimiento giratorio dentro de geometrías variables

# Celda 1: Instalación de paquetes y configuración inicial
# Ejecutar esta celda primero
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
from scipy.integrate import odeint
import time
from IPython.display import clear_output

# Configuración de visualización para Colab
%matplotlib inline
plt.rcParams['figure.figsize'] = [12, 8]

print("Configuración completada. Paquetes importados correctamente.")

# Celda 2: Definición de la clase FluidSimulator
class FluidSimulator:
    """
    Simulador CFD para fluidos en movimiento giratorio dentro de geometrías variables
    que pueden transformarse desde esferas a toroides, con un eje central conductor.
    """

    def __init__(self, grid_size=30, sphere_radius=1.0, time_step=0.01, viscosity=0.1):
        """
        Inicializa el simulador con parámetros básicos

        Args:
            grid_size: Número de puntos en cada dimensión
            sphere_radius: Radio de la esfera límite externa
            time_step: Paso de tiempo para la simulación
            viscosity: Viscosidad cinemática del fluido
        """
        self.grid_size = grid_size
        self.sphere_radius = sphere_radius
        self.time_step = time_step
        self.viscosity = viscosity

        # Crear mallas para coordenadas
        self.x = np.linspace(-sphere_radius, sphere_radius, grid_size)
        self.y = np.linspace(-sphere_radius, sphere_radius, grid_size)
        self.z = np.linspace(-sphere_radius, sphere_radius, grid_size)
        self.X, self.Y, self.Z = np.meshgrid(self.x, self.y, self.z, indexing='ij')

        # Campos iniciales de velocidad (U, V, W) y presión (P)
        self.U = np.zeros((grid_size, grid_size, grid_size))
        self.V = np.zeros((grid_size, grid_size, grid_size))
        self.W = np.zeros((grid_size, grid_size, grid_size))
        self.P = np.zeros((grid_size, grid_size, grid_size))

        # Definir geometría inicial (cilíndrica con base plana)
        self.container_type = "cylinder"
        self.inner_radius = 0.4 * sphere_radius
        self.outer_radius = 0.8 * sphere_radius
        self.cylinder_height = 1.6 * sphere_radius
        self.toroid_factor = 0.0  # Factor para transformación a toroide (0-1)

        # Propiedades del eje central
        self.central_axis_radius = 0.05 * sphere_radius
        self.central_axis_rotation = 0.0  # Velocidad angular del eje (rad/s)

        # Crear máscara inicial para determinar qué celdas están dentro del dominio
        self.update_geometry_mask()

        # Estado del sistema
        self.time = 0.0
        self.iteration = 0

    def update_geometry_mask(self):
        """Actualiza la máscara de geometría según la configuración actual"""
        # Coordenadas radiales y altura
        R = np.sqrt(self.X**2 + self.Y**2)

        # Máscara para la esfera externa límite
        sphere_mask = self.X**2 + self.Y**2 + self.Z**2 <= self.sphere_radius**2

        # Máscara para el eje central
        central_axis_mask = R <= self.central_axis_radius

        # Inicialmente todas las celdas están fuera del dominio
        self.mask = np.zeros((self.grid_size, self.grid_size, self.grid_size), dtype=bool)

        if self.container_type == "cylinder":
            # Cilindro: R <= radio_cilindro y -altura/2 <= Z <= altura/2
            cylinder_mask = (R <= self.inner_radius) & (np.abs(self.Z) <= self.cylinder_height/2)
            self.mask = cylinder_mask & sphere_mask & ~central_axis_mask

        elif self.container_type == "sphere":
            # Esfera interior
            inner_sphere_mask = self.X**2 + self.Y**2 + self.Z**2 <= self.inner_radius**2
            self.mask = inner_sphere_mask & ~central_axis_mask

        elif self.container_type == "toroid":
            # Radio mayor del toroide
            major_radius = (self.inner_radius + self.outer_radius) / 2
            # Radio menor del toroide
            minor_radius = (self.outer_radius - self.inner_radius) / 2

            # Distancia desde el punto al círculo central del toroide en el plano XY
            dist_from_circle = np.abs(np.sqrt(self.X**2 + self.Y**2) - major_radius)

            # Máscara del toroide
            toroid_mask = dist_from_circle**2 + self.Z**2 <= minor_radius**2
            self.mask = toroid_mask & sphere_mask & ~central_axis_mask

        elif self.container_type == "hybrid":
            # Interpolación entre cilindro y toroide
            R_major = (self.inner_radius + self.outer_radius) / 2
            R_minor = (self.outer_radius - self.inner_radius) / 2

            # Factor de ponderación para la transformación
            t = self.toroid_factor

            # Para t=0: cilindro, para t=1: toroide
            if t <= 0:
                cylinder_mask = (R <= self.inner_radius) & (np.abs(self.Z) <= self.cylinder_height/2)
                self.mask = cylinder_mask & sphere_mask & ~central_axis_mask
            elif t >= 1:
                dist_from_circle = np.abs(np.sqrt(self.X**2 + self.Y**2) - R_major)
                toroid_mask = dist_from_circle**2 + self.Z**2 <= R_minor**2
                self.mask = toroid_mask & sphere_mask & ~central_axis_mask
            else:
                # Interpolación de la geometría
                cylinder_center_dist = np.maximum(
                    np.abs(R - self.inner_radius/2),
                    np.abs(self.Z) - self.cylinder_height/2
                )
                dist_from_circle = np.abs(np.sqrt(self.X**2 + self.Y**2) - R_major)
                toroid_dist = np.sqrt(dist_from_circle**2 + self.Z**2) - R_minor

                # Combinar con interpolación suave
                hybrid_dist = (1 - t) * cylinder_center_dist + t * toroid_dist
                hybrid_mask = hybrid_dist <= 0
                self.mask = hybrid_mask & sphere_mask & ~central_axis_mask

    def set_container_type(self, container_type, toroid_factor=None):
        """
        Establece el tipo de contenedor para la simulación

        Args:
            container_type: "cylinder", "sphere", "toroid", o "hybrid"
            toroid_factor: Factor de transformación para "hybrid" (0-1)
        """
        self.container_type = container_type
        if toroid_factor is not None:
            self.toroid_factor = max(0.0, min(1.0, toroid_factor))
        self.update_geometry_mask()

    def set_rotation_speed(self, speed):
        """Establece la velocidad de rotación del eje central"""
        self.central_axis_rotation = speed

    def initialize_fluid_state(self, init_mode="rest"):
        """
        Inicializa el estado del fluido

        Args:
            init_mode: Modo de inicialización ("rest", "rigid_rotation", "random")
        """
        # Reiniciar campos
        self.U = np.zeros((self.grid_size, self.grid_size, self.grid_size))
        self.V = np.zeros((self.grid_size, self.grid_size, self.grid_size))
        self.W = np.zeros((self.grid_size, self.grid_size, self.grid_size))
        self.P = np.zeros((self.grid_size, self.grid_size, self.grid_size))

        if init_mode == "rest":
            # Fluido en reposo, ya inicializado a cero
            pass

        elif init_mode == "rigid_rotation":
            # Inicializar con rotación rígida
            omega = 1.0  # Velocidad angular inicial
            for i in range(self.grid_size):
                for j in range(self.grid_size):
                    for k in range(self.grid_size):
                        if self.mask[i, j, k]:
                            # Componentes de velocidad tangencial
                            self.U[i, j, k] = -omega * self.Y[i, j, k]
                            self.V[i, j, k] = omega * self.X[i, j, k]

        elif init_mode == "random":
            # Inicializar con pequeñas perturbaciones aleatorias
            noise_amplitude = 0.05
            for i in range(self.grid_size):
                for j in range(self.grid_size):
                    for k in range(self.grid_size):
                        if self.mask[i, j, k]:
                            self.U[i, j, k] = noise_amplitude * (np.random.rand() - 0.5)
                            self.V[i, j, k] = noise_amplitude * (np.random.rand() - 0.5)
                            self.W[i, j, k] = noise_amplitude * (np.random.rand() - 0.5)

    def apply_boundary_conditions(self):
        """Aplica condiciones de frontera a los campos de velocidad"""
        # Condición de no deslizamiento en las paredes
        for i in range(self.grid_size):
            for j in range(self.grid_size):
                for k in range(self.grid_size):
                    if not self.mask[i, j, k]:
                        # Fuera del dominio del fluido, fijar velocidad a cero
                        self.U[i, j, k] = 0
                        self.V[i, j, k] = 0
                        self.W[i, j, k] = 0

        # Condición especial en el eje central (velocidad tangencial)
        R = np.sqrt(self.X**2 + self.Y**2)
        axis_mask = R <= self.central_axis_radius

        for i in range(self.grid_size):
            for j in range(self.grid_size):
                for k in range(self.grid_size):
                    if axis_mask[i, j, k] and (self.X[i,j,k]**2 + self.Y[i,j,k]**2 + self.Z[i,j,k]**2 <= self.sphere_radius**2):
                        # Aplicar rotación en el eje central
                        self.U[i, j, k] = -self.central_axis_rotation * self.Y[i, j, k]
                        self.V[i, j, k] = self.central_axis_rotation * self.X[i, j, k]
                        self.W[i, j, k] = 0

    def compute_velocity_gradient(self):
        """Calcula los gradientes de velocidad"""
        dx = self.x[1] - self.x[0]
        dy = self.y[1] - self.y[0]
        dz = self.z[1] - self.z[0]

        # Derivadas parciales usando diferencias finitas centradas
        dUdx = np.zeros_like(self.U)
        dUdy = np.zeros_like(self.U)
        dUdz = np.zeros_like(self.U)
        dVdx = np.zeros_like(self.V)
        dVdy = np.zeros_like(self.V)
        dVdz = np.zeros_like(self.V)
        dWdx = np.zeros_like(self.W)
        dWdy = np.zeros_like(self.W)
        dWdz = np.zeros_like(self.W)

        # Calcular gradientes en puntos interiores
        for i in range(1, self.grid_size - 1):
            for j in range(1, self.grid_size - 1):
                for k in range(1, self.grid_size - 1):
                    if self.mask[i, j, k]:
                        dUdx[i, j, k] = (self.U[i+1, j, k] - self.U[i-1, j, k]) / (2*dx)
                        dUdy[i, j, k] = (self.U[i, j+1, k] - self.U[i, j-1, k]) / (2*dy)
                        dUdz[i, j, k] = (self.U[i, j, k+1] - self.U[i, j, k-1]) / (2*dz)

                        dVdx[i, j, k] = (self.V[i+1, j, k] - self.V[i-1, j, k]) / (2*dx)
                        dVdy[i, j, k] = (self.V[i, j+1, k] - self.V[i, j-1, k]) / (2*dy)
                        dVdz[i, j, k] = (self.V[i, j, k+1] - self.V[i, j, k-1]) / (2*dz)

                        dWdx[i, j, k] = (self.W[i+1, j, k] - self.W[i-1, j, k]) / (2*dx)
                        dWdy[i, j, k] = (self.W[i, j+1, k] - self.W[i, j-1, k]) / (2*dy)
                        dWdz[i, j, k] = (self.W[i, j, k+1] - self.W[i, j, k-1]) / (2*dz)

        return dUdx, dUdy, dUdz, dVdx, dVdy, dVdz, dWdx, dWdy, dWdz

    def solve_pressure_poisson(self):
        """Resuelve la ecuación de Poisson para la presión usando Jacobi"""
        dx = self.x[1] - self.x[0]
        dy = self.y[1] - self.y[0]
        dz = self.z[1] - self.z[0]

        # Divergencia del campo de velocidad
        dUdx, dUdy, dUdz, dVdx, dVdy, dVdz, dWdx, dWdy, dWdz = self.compute_velocity_gradient()
        div = dUdx + dVdy + dWdz

        # Término fuente para la ecuación de Poisson
        rhs = div / self.time_step

        # Matriz inicial de presión
        p_new = np.copy(self.P)

        # Número de iteraciones para la solución de Poisson
        n_iter = 50

        # Resolver mediante el método iterativo de Jacobi
        for _ in range(n_iter):
            for i in range(1, self.grid_size - 1):
                for j in range(1, self.grid_size - 1):
                    for k in range(1, self.grid_size - 1):
                        if self.mask[i, j, k]:
                            p_new[i, j, k] = (1/6) * (
                                self.P[i+1, j, k] + self.P[i-1, j, k] +
                                self.P[i, j+1, k] + self.P[i, j-1, k] +
                                self.P[i, j, k+1] + self.P[i, j, k-1] -
                                dx**2 * rhs[i, j, k]
                            )

            # Actualizar campo de presión
            self.P = np.copy(p_new)

        return self.P

    def update_velocity_field(self):
        """Actualiza el campo de velocidad utilizando el esquema de proyección"""
        dx = self.x[1] - self.x[0]
        dy = self.y[1] - self.y[0]
        dz = self.z[1] - self.z[0]

        # Calcular gradientes de velocidad
        dUdx, dUdy, dUdz, dVdx, dVdy, dVdz, dWdx, dWdy, dWdz = self.compute_velocity_gradient()

        # Campos temporales para almacenar velocidades intermedias
        u_temp = np.copy(self.U)
        v_temp = np.copy(self.V)
        w_temp = np.copy(self.W)

        # Calcular campos intermedios (paso de advección-difusión)
        for i in range(1, self.grid_size - 1):
            for j in range(1, self.grid_size - 1):
                for k in range(1, self.grid_size - 1):
                    if self.mask[i, j, k]:
                        # Términos advectivos (esquema upwind)
                        if self.U[i, j, k] > 0:
                            du_dx = (self.U[i, j, k] - self.U[i-1, j, k]) / dx
                        else:
                            du_dx = (self.U[i+1, j, k] - self.U[i, j, k]) / dx

                        if self.V[i, j, k] > 0:
                            du_dy = (self.U[i, j, k] - self.U[i, j-1, k]) / dy
                        else:
                            du_dy = (self.U[i, j+1, k] - self.U[i, j, k]) / dy

                        if self.W[i, j, k] > 0:
                            du_dz = (self.U[i, j, k] - self.U[i, j, k-1]) / dz
                        else:
                            du_dz = (self.U[i, j, k+1] - self.U[i, j, k]) / dz

                        # Términos similares para V y W
                        if self.U[i, j, k] > 0:
                            dv_dx = (self.V[i, j, k] - self.V[i-1, j, k]) / dx
                        else:
                            dv_dx = (self.V[i+1, j, k] - self.V[i, j, k]) / dx

                        if self.V[i, j, k] > 0:
                            dv_dy = (self.V[i, j, k] - self.V[i, j-1, k]) / dy
                        else:
                            dv_dy = (self.V[i, j+1, k] - self.V[i, j, k]) / dy

                        if self.W[i, j, k] > 0:
                            dv_dz = (self.V[i, j, k] - self.V[i, j, k-1]) / dz
                        else:
                            dv_dz = (self.V[i, j, k+1] - self.V[i, j, k]) / dz

                        if self.U[i, j, k] > 0:
                            dw_dx = (self.W[i, j, k] - self.W[i-1, j, k]) / dx
                        else:
                            dw_dx = (self.W[i+1, j, k] - self.W[i, j, k]) / dx

                        if self.V[i, j, k] > 0:
                            dw_dy = (self.W[i, j, k] - self.W[i, j-1, k]) / dy
                        else:
                            dw_dy = (self.W[i, j+1, k] - self.W[i, j, k]) / dy

                        if self.W[i, j, k] > 0:
                            dw_dz = (self.W[i, j, k] - self.W[i, j, k-1]) / dz
                        else:
                            dw_dz = (self.W[i, j, k+1] - self.W[i, j, k]) / dz

                        # Términos difusivos (Laplaciano)
                        laplacian_u = (
                            self.U[i+1, j, k] - 2*self.U[i, j, k] + self.U[i-1, j, k]
                        ) / dx**2 + (
                            self.U[i, j+1, k] - 2*self.U[i, j, k] + self.U[i, j-1, k]
                        ) / dy**2 + (
                            self.U[i, j, k+1] - 2*self.U[i, j, k] + self.U[i, j, k-1]
                        ) / dz**2

                        laplacian_v = (
                            self.V[i+1, j, k] - 2*self.V[i, j, k] + self.V[i-1, j, k]
                        ) / dx**2 + (
                            self.V[i, j+1, k] - 2*self.V[i, j, k] + self.V[i, j-1, k]
                        ) / dy**2 + (
                            self.V[i, j, k+1] - 2*self.V[i, j, k] + self.V[i, j, k-1]
                        ) / dz**2

                        laplacian_w = (
                            self.W[i+1, j, k] - 2*self.W[i, j, k] + self.W[i-1, j, k]
                        ) / dx**2 + (
                            self.W[i, j+1, k] - 2*self.W[i, j, k] + self.W[i, j-1, k]
                        ) / dy**2 + (
                            self.W[i, j, k+1] - 2*self.W[i, j, k] + self.W[i, j, k-1]
                        ) / dz**2

                        # Actualizar velocidades intermedias (ecuación de Navier-Stokes sin término de presión)
                        u_temp[i, j, k] = self.U[i, j, k] - self.time_step * (
                            self.U[i, j, k] * du_dx +
                            self.V[i, j, k] * du_dy +
                            self.W[i, j, k] * du_dz
                        ) + self.time_step * self.viscosity * laplacian_u

                        v_temp[i, j, k] = self.V[i, j, k] - self.time_step * (
                            self.U[i, j, k] * dv_dx +
                            self.V[i, j, k] * dv_dy +
                            self.W[i, j, k] * dv_dz
                        ) + self.time_step * self.viscosity * laplacian_v

                        w_temp[i, j, k] = self.W[i, j, k] - self.time_step * (
                            self.U[i, j, k] * dw_dx +
                            self.V[i, j, k] * dw_dy +
                            self.W[i, j, k] * dw_dz
                        ) + self.time_step * self.viscosity * laplacian_w

        # Actualizar campos temporales
        self.U = u_temp
        self.V = v_temp
        self.W = w_temp

        # Resolver ecuación de Poisson para la presión
        self.solve_pressure_poisson()

        # Proyectar el campo de velocidad para hacerlo libre de divergencia
        for i in range(1, self.grid_size - 1):
            for j in range(1, self.grid_size - 1):
                for k in range(1, self.grid_size - 1):
                    if self.mask[i, j, k]:
                        # Gradiente de presión
                        dp_dx = (self.P[i+1, j, k] - self.P[i-1, j, k]) / (2*dx)
                        dp_dy = (self.P[i, j+1, k] - self.P[i, j-1, k]) / (2*dy)
                        dp_dz = (self.P[i, j, k+1] - self.P[i, j, k-1]) / (2*dz)

                        # Corrección del campo de velocidad
                        self.U[i, j, k] -= self.time_step * dp_dx
                        self.V[i, j, k] -= self.time_step * dp_dy
                        self.W[i, j, k] -= self.time_step * dp_dz

    def simulate_step(self):
        """Ejecuta un paso de simulación"""
        # Aplicar condiciones de frontera
        self.apply_boundary_conditions()

        # Actualizar campo de velocidad
        self.update_velocity_field()

        # Incrementar el tiempo y la iteración
        self.time += self.time_step
        self.iteration += 1

        return self.U, self.V, self.W, self.P

    def run_simulation(self, n_steps):
        """
        Ejecuta la simulación por un número determinado de pasos

        Args:
            n_steps: Número de pasos a simular

        Returns:
            Diccionario con resultados finales
        """
        results = {
            'iterations': [],
            'time': [],
            'kinetic_energy': [],
            'enstrophy': []
        }

        for step in range(n_steps):
            # Ejecutar paso de simulación
            self.simulate_step()

            # Calcular energía cinética y enstrofía
            ke = self.calculate_kinetic_energy()
            ens = self.calculate_enstrophy()

            # Almacenar resultados
            results['iterations'].append(self.iteration)
            results['time'].append(self.time)
            results['kinetic_energy'].append(ke)
            results['enstrophy'].append(ens)

            # Mostrar progreso cada 10 pasos
            if step % 10 == 0:
                print(f"Paso {step}/{n_steps}, t={self.time:.3f}, KE={ke:.6f}, Ens={ens:.6f}")

        return results

    def calculate_kinetic_energy(self):
        """Calcula la energía cinética total del fluido"""
        ke = 0.0
        n_cells = 0

        for i in range(self.grid_size):
            for j in range(self.grid_size):
                for k in range(self.grid_size):
                    if self.mask[i, j, k]:
                        u2 = self.U[i, j, k]**2
                        v2 = self.V[i, j, k]**2
                        w2 = self.W[i, j, k]**2
                        ke += 0.5 * (u2 + v2 + w2)
                        n_cells += 1

        return ke / max(1, n_cells)

    def calculate_enstrophy(self):
        """Calcula la enstrofía (suma de vorticidad al cuadrado)"""
        # Gradientes de velocidad
        dUdx, dUdy, dUdz, dVdx, dVdy, dVdz, dWdx, dWdy, dWdz = self.compute_velocity_gradient()

        # Componentes de vorticidad
        wx = dWdy - dVdz  # ∂w/∂y - ∂v/∂z
        wy = dUdz - dWdx  # ∂u/∂z - ∂w/∂x
        wz = dVdx - dUdy  # ∂v/∂x - ∂u/∂y

        # Calcular enstrofía
        enstrophy = 0.0
        n_cells = 0

        for i in range(1, self.grid_size - 1):
            for j in range(1, self.grid_size - 1):
                for k in range(1, self.grid_size - 1):
                    if self.mask[i, j, k]:
                        enstrophy += 0.5 * (wx[i, j, k]**2 + wy[i, j, k]**2 + wz[i, j, k]**2)
                        n_cells += 1

        return enstrophy / max(1, n_cells)

    def visualize_flow(self, slice_z=None, streamlines=True, quiver=True, vorticity=False):
        """
        Visualiza el flujo en un corte 2D

        Args:
            slice_z: Índice del corte en z (si es None, se usa el centro)
            streamlines: Mostrar líneas de corriente
            quiver: Mostrar vectores de velocidad
            vorticity: Mostrar la magnitud de vorticidad
        """
        if slice_z is None:
            slice_z = self.grid_size // 2

        # Crear figura
        fig, ax = plt.subplots(figsize=(10, 8))

        # Obtener malla 2D para el corte
        X2D, Y2D = np.meshgrid(self.x, self.y)

        # Extraer campos de velocidad en el corte
        U2D = self.U[:, :, slice_z]
        V2D = self.V[:, :, slice_z]

        # Extraer máscara para el dominio en el corte
        mask2D = self.mask[:, :, slice_z]

        # Aplicar máscara a velocidades (poner a cero fuera del dominio)
        U2D_masked = np.where(mask2D, U2D, 0)
        V2D_masked = np.where(mask2D, V2D, 0)

        # Magnitud de velocidad
        speed = np.sqrt(U2D_masked**2 + V2D_masked**2)

        # Mostrar contorno del dominio
        ax.contour(X2D, Y2D, mask2D.astype(int), levels=[0.5], colors='k', linewidths=2)

        # Mostrar campo escalar (velocidad o vorticidad)
        if vorticity:
            # Calcular vorticidad en el plano (solo componente z)
            dUdx = np.zeros_like(U2D)
            dUdy = np.zeros_like(U2D)
            dVdx = np.zeros_like(V2D)
            dVdy = np.zeros_like(V2D)

            dx = self.x[1] - self.x[0]
            dy = self.y[1] - self.y[0]

            for i in range(1, self.grid_size - 1):
                for j in range(1, self.grid_size - 1):
                    if mask2D[i, j]:
                        dUdx[i, j] = (U2D[i+1, j] - U2D[i-1, j]) / (2*dx)
                        dUdy[i, j] = (U2D[i, j+1] - U2D[i, j-1]) / (2*dy)
                        dVdx[i, j] = (V2D[i+1, j] - V2D[i-1, j]) / (2*dx)
                        dVdy[i, j] = (V2D[i, j+1] - V2D[i, j-1]) / (2*dy)

            vort_z = dVdx - dUdy
            scalar_field = np.ma.masked_where(~mask2D, vort_z)
            title = f"Vorticidad en z={self.z[slice_z]:.2f}, t={self.time:.3f}"
        else:
            scalar_field = np.ma.masked_where(~mask2D, speed)
            title = f"Velocidad en z={self.z[slice_z]:.2f}, t={self.time:.3f}"

        # Mostrar campo escalar
        cf = ax.pcolormesh(X2D, Y2D, scalar_field, cmap='viridis', shading='auto')
        plt.colorbar(cf, ax=ax)

        # Mostrar líneas de corriente
        if streamlines:
            ax.streamplot(X2D.T, Y2D.T, U2D_masked.T, V2D_masked.T,
                         color='white', density=1.5, linewidth=0.5, arrowsize=0.8)

        # Mostrar vectores de velocidad
        if quiver:
            # Reducir la densidad de vectores para mayor claridad
            stride = 2
            ax.quiver(X2D[::stride, ::stride], Y2D[::stride, ::stride],
                     U2D_masked[::stride, ::stride], V2D_masked[::stride, ::stride],
                     color='k', scale=20, width=0.003)

        ax.set_title(title)
        ax.set_xlabel('X')
        ax.set_ylabel('Y')
        ax.set_aspect('equal')

        plt.tight_layout()
        return fig, ax


# Celda 3: Definición de la clase CFDSimulationDemo para la interfaz interactiva
class CFDSimulationDemo:
    """Clase para demostrar y controlar la simulación de fluidos"""

    def __init__(self):
        """Inicializa la demostración"""
        # Parámetros predeterminados
        self.grid_size = 32  # Reducido para mejor rendimiento en Colab
        self.sphere_radius = 1.0
        self.time_step = 0.01
        self.viscosity = 0.01

        # Crear el simulador
        self.simulator = FluidSimulator(
            grid_size=self.grid_size,
            sphere_radius=self.sphere_radius,
            time_step=self.time_step,
            viscosity=self.viscosity
        )

        # Inicializar el fluido
        self.simulator.initialize_fluid_state(init_mode="rigid_rotation")

        # Configurar geometría inicial
        self.simulator.set_container_type("cylinder")

        # Historial de resultados
        self.results = {
            'iterations': [],
            'time': [],
            'kinetic_energy': [],
            'enstrophy': []
        }

    def update_parameters(self, time_step=None, viscosity=None):
        """Actualiza los parámetros de simulación"""
        if time_step is not None:
            self.simulator.time_step = time_step

        if viscosity is not None:
            self.simulator.viscosity = viscosity

    def change_geometry(self, container_type, toroid_factor=None):
        """Cambia la geometría del contenedor"""
        self.simulator.set_container_type(container_type, toroid_factor)

    def set_rotation_speed(self, speed):
        """Establece la velocidad de rotación del eje central"""
        self.simulator.set_rotation_speed(speed)

    def run_simulation(self, n_steps):
        """Ejecuta la simulación por un número de pasos"""
        results = self.simulator.run_simulation(n_steps)

        # Actualizar historial de resultados
        for key in results:
            self.results[key].extend(results[key])

    def plot_results(self):
        """Grafica los resultados de la simulación"""
        # Crear figura con dos subtramas
        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8))

        # Graficar energía cinética
        ax1.plot(self.results['time'], self.results['kinetic_energy'], 'b-')
        ax1.set_xlabel('Tiempo')
        ax1.set_ylabel('Energía Cinética')
        ax1.set_title('Evolución de la Energía Cinética')
        ax1.grid(True)

        # Graficar enstrofía
        ax2.plot(self.results['time'], self.results['enstrophy'], 'r-')
        ax2.set_xlabel('Tiempo')
        ax2.set_ylabel('Enstrofía')
        ax2.set_title('Evolución de la Enstrofía')
        ax2.grid(True)

        plt.tight_layout()
        return fig

    def visualize_current_state(self, view_3d=False):
        """Visualiza el estado actual de la simulación"""
        if view_3d:
            return self.simulator.visualize_3d()
        else:
            return self.simulator.visualize_flow()

    def run_demo(self):
        """Ejecuta una demostración completa"""
        print("Iniciando simulación con geometría cilíndrica...")
        self.run_simulation(20)  # Reducido para mejor rendimiento

        print("\nCambiando a geometría híbrida (transformación hacia toroide)...")
        self.change_geometry("hybrid", toroid_factor=0.5)
        self.run_simulation(20)

        print("\nAumentando velocidad de rotación del eje central...")
        self.set_rotation_speed(5.0)
        self.run_simulation(20)

        print("\nCambiando a geometría toroidal completa...")
        self.change_geometry("toroid")
        self.run_simulation(20)

        print("\nSimulación completada. Generando visualizaciones...")

        # Visualizar resultados
        fig_results = self.plot_results()
        plt.show()

        fig_2d, _ = self.visualize_current_state(view_3d=False)
        plt.show()

        fig_3d, _ = self.visualize_current_state(view_3d=True)
        plt.show()

    def run_interactive_simulation(self, n_steps=5, update_freq=1):
        """Ejecuta una simulación interactiva con visualización en tiempo real"""
        # Configurar figura inicial
        fig, ax = plt.subplots(figsize=(10, 8))

        for step in range(n_steps):
            # Ejecutar n pasos de simulación
            for _ in range(update_freq):
                self.simulator.simulate_step()

            # Limpiar eje para la nueva visualización
            ax.clear()

            # Obtener corte 2D en el medio
            slice_z = self.grid_size // 2
            X2D, Y2D = np.meshgrid(self.simulator.x, self.simulator.y)

            # Extraer campos de velocidad en el corte
            U2D = self.simulator.U[:, :, slice_z]
            V2D = self.simulator.V[:, :, slice_z]

            # Extraer máscara para el dominio en el corte
            mask2D = self.simulator.mask[:, :, slice_z]

            # Aplicar máscara a velocidades
            U2D_masked = np.where(mask2D, U2D, 0)
            V2D_masked = np.where(mask2D, V2D, 0)

            # Magnitud de velocidad
            speed = np.sqrt(U2D_masked**2 + V2D_masked**2)

            # Mostrar contorno del dominio
            ax.contour(X2D, Y2D, mask2D.astype(int), levels=[0.5], colors='k', linewidths=2)

            # Mostrar campo de velocidad
            scalar_field = np.ma.masked_where(~mask2D, speed)
            cf = ax.pcolormesh(X2D, Y2D, scalar_field, cmap='viridis', shading='auto')
            plt.colorbar(cf, ax=ax)

            # Mostrar líneas de corriente
            ax.streamplot(X2D.T, Y2D.T, U2D_masked.T, V2D_masked.T,
                         color='white', density=1.5, linewidth=0.5, arrowsize=0.8)

            # Actualizar título con información del tiempo actual
            ax.set_title(f"Simulación en tiempo real, t={self.simulator.time:.3f}, Paso {step+1}/{n_steps}")
            ax.set_xlabel('X')
            ax.set_ylabel('Y')
            ax.set_aspect('equal')

            # Mostrar la figura actualizada
            plt.tight_layout()
            display.clear_output(wait=True)
            display.display(plt.gcf())

            # Pequeña pausa para visualización
            time.sleep(0.1)

        # Mostrar mensaje de finalización
        print(f"Simulación completada: {n_steps} pasos, tiempo final: {self.simulator.time:.3f}")

        # Actualizar historial de resultados
        ke = self.simulator.calculate_kinetic_energy()
        ens = self.simulator.calculate_enstrophy()
        self.results['iterations'].append(self.simulator.iteration)
        self.results['time'].append(self.simulator.time)
        self.results['kinetic_energy'].append(ke)
        self.results['enstrophy'].append(ens)


# Celda 4: Interfaz interactiva con widgets para controlar la simulación
# Esta celda crea controles interactivos para manipular la simulación
try:
    from ipywidgets import interact, interactive, fixed, widgets
    from IPython.display import display, clear_output

    def run_interactive_demo():
        # Crear instancia de la demo
        demo = CFDSimulationDemo()

        # Función para actualizar parámetros
        def update_params(container_type, toroid_factor, rotation_speed, viscosity):
            demo.change_geometry(container_type, toroid_factor)
            demo.set_rotation_speed(rotation_speed)
            demo.update_parameters(viscosity=viscosity)

            # Mostrar configuración actual
            print(f"Configuración actual:")
            print(f"- Tipo de contenedor: {container_type}")
            print(f"- Factor toroide: {toroid_factor:.2f}")
            print(f"- Velocidad de rotación: {rotation_speed:.2f}")
            print(f"- Viscosidad: {viscosity:.4f}")

        # Función para ejecutar simulación con los parámetros actuales
        def run_simulation(n_steps=10):
            clear_output(wait=True)
            print(f"Ejecutando simulación con {n_steps} pasos...")
            demo.run_interactive_simulation(n_steps=n_steps)

            # Mostrar resultados
            fig_results = demo.plot_results()
            plt.show()

        # Crear widgets interactivos
        container_widget = widgets.Dropdown(
            options=['cylinder', 'sphere', 'toroid', 'hybrid'],
            value='cylinder',
            description='Contenedor:'
        )

        toroid_factor_widget = widgets.FloatSlider(
            value=0.0,
            min=0.0,
            max=1.0,
            step=0.1,
            description='Factor toroide:',
            disabled=False
        )

        rotation_widget = widgets.FloatSlider(
            value=1.0,
            min=0.0,
            max=10.0,
            step=0.5,
            description='Rotación:',
            disabled=False
        )

        viscosity_widget = widgets.FloatLogSlider(
            value=0.01,
            base=10,
            min=-3,  # 10^-3
            max=0,   # 10^0
            step=0.1,
            description='Viscosidad:',
            disabled=False
        )

        steps_widget = widgets.IntSlider(
            value=10,
            min=5,
            max=50,
            step=5,
            description='Pasos:',
            disabled=False
        )

        # Crear botones
        update_button = widgets.Button(description="Actualizar Parámetros")
        run_button = widgets.Button(description="Ejecutar Simulación")
        reset_button = widgets.Button(description="Reiniciar Simulación")

        # Definir acciones de los botones
        def on_update_clicked(b):
            update_params(
                container_widget.value,
                toroid_factor_widget.value,
                rotation_widget.value,
                viscosity_widget.value
            )

        def on_run_clicked(b):
            run_simulation(steps_widget.value)

        def on_reset_clicked(b):
            # Crear una nueva instancia de la demo
            nonlocal demo
            demo = CFDSimulationDemo()
            clear_output(wait=True)
            print("Simulación reiniciada con valores predeterminados.")

        # Asignar acciones a los botones
        update_button.on_click(on_update_clicked)
        run_button.on_click(on_run_clicked)
        reset_button.on_click(on_reset_clicked)

        # Mostrar widgets
        print("Interfaz Interactiva para Simulación CFD")
        print("----------------------------------------")
        print("1. Configura los parámetros de simulación")
        print("2. Haz clic en 'Actualizar Parámetros'")
        print("3. Haz clic en 'Ejecutar Simulación'")

        display(container_widget)
        display(toroid_factor_widget)
        display(rotation_widget)
        display(viscosity_widget)
        display(steps_widget)

        button_box = widgets.HBox([update_button, run_button, reset_button])
        display(button_box)

    # Esta función se ejecutará cuando se ejecute la celda
    def setup_interactive_interface():
        print("Configurando interfaz interactiva...")
        run_interactive_demo()

except ImportError:
    print("Para usar la interfaz interactiva, instala ipywidgets:")
    print("!pip install ipywidgets")
    print("Y luego reinicia el kernel de Jupyter")


# Celda 5: Ejemplos de ejecución de simulaciones
# Aquí puedes ejecutar diferentes configuraciones de simulación

# Función para ejecutar una simulación básica
def run_basic_simulation():
    """Ejecuta una simulación básica con visualizaciones"""
    # Crear simulador con parámetros personalizados
    simulator = FluidSimulator(
        grid_size=30,          # Resolución de la malla
        sphere_radius=1.0,     # Radio de la esfera límite
        time_step=0.01,        # Paso de tiempo
        viscosity=0.01         # Viscosidad del fluido
    )

    # Inicializar el fluido con rotación rígida
    simulator.initialize_fluid_state(init_mode="rigid_rotation")

    # Configurar geometría cilíndrica
    simulator.set_container_type("cylinder")

    # Establecer velocidad de rotación del eje central
    simulator.set_rotation_speed(3.0)

    # Ejecutar simulación por 30 pasos
    print("Ejecutando simulación básica...")
    simulator.run_simulation(30)

    # Visualizar resultados
    print("Generando visualizaciones...")

    # Visualización 2D
    fig_2d, _ = simulator.visualize_flow(streamlines=True, quiver=True)
    plt.show()

    # Visualización 3D
    fig_3d, _ = simulator.visualize_3d(threshold=0.1)
    plt.show()

    return simulator

# Función para ejecutar una simulación completa de transformación de geometría
def run_transformation_simulation():
    """Ejecuta una simulación que muestra la transformación de geometría"""
    # Crear instancia de la demo
    demo = CFDSimulationDemo()

    # Ejecutar secuencia de simulaciones con diferentes geometrías
    print("Fase 1: Geometría cilíndrica")
    demo.run_simulation(10)

    # Visualizar estado actual
    fig, _ = demo.visualize_current_state()
    plt.show()

    print("Fase 2: Transformación a geometría híbrida (25%)")
    demo.change_geometry("hybrid", toroid_factor=0.25)
    demo.run_simulation(10)

    # Visualizar estado actual
    fig, _ = demo.visualize_current_state()
    plt.show()

    print("Fase 3: Transformación a geometría híbrida (50%)")
    demo.change_geometry("hybrid", toroid_factor=0.5)
    demo.run_simulation(10)

    # Visualizar estado actual
    fig, _ = demo.visualize_current_state()
    plt.show()

    print("Fase 4: Transformación a geometría híbrida (75%)")
    demo.change_geometry("hybrid", toroid_factor=0.75)
    demo.run_simulation(10)

    # Visualizar estado actual
    fig, _ = demo.visualize_current_state()
    plt.show()

    print("Fase 5: Geometría toroidal completa")
    demo.change_geometry("toroid")
    demo.run_simulation(10)

    # Visualizar estado final
    fig, _ = demo.visualize_current_state()
    plt.show()

    # Visualizar resultados acumulados
    fig_results = demo.plot_results()
    plt.show()

    # Visualización 3D del estado final
    fig_3d, _ = demo.visualize_current_state(view_3d=True)
    plt.show()

    return demo

# Función para ejecutar una simulación con diferentes velocidades de rotación
def run_rotation_speed_test():
    """Ejecuta una simulación que prueba diferentes velocidades de rotación"""
    # Crear simulador
    simulator = FluidSimulator(
        grid_size=30,
        sphere_radius=1.0,
        time_step=0.01,
        viscosity=0.01
    )

    # Inicializar con fluido en reposo
    simulator.initialize_fluid_state(init_mode="rest")

    # Configurar geometría esférica
    simulator.set_container_type("sphere")

    # Lista de velocidades de rotación a probar
    rotation_speeds = [0.0, 1.0, 3.0, 5.0, 10.0]

    # Almacenar resultados
    results = []

    for speed in rotation_speeds:
        print(f"Probando velocidad de rotación: {speed}")

        # Establecer velocidad de rotación
        simulator.set_rotation_speed(speed)

        # Ejecutar simulación
        simulator.run_simulation(20)

        # Calcular métricas
        ke = simulator.calculate_kinetic_energy()
        ens = simulator.calculate_enstrophy()

        # Almacenar resultados
        results.append({
            'speed': speed,
            'time': simulator.time,
            'kinetic_energy': ke,
            'enstrophy': ens
        })

        # Visualizar
        fig, _ = simulator.visualize_flow(streamlines=True, vorticity=True)
        plt.title(f"Velocidad de rotación: {speed}, t={simulator.time:.3f}")
        plt.show()

    # Graficar resultados comparativos
    speeds = [r['speed'] for r in results]
    ke_values = [r['kinetic_energy'] for r in results]
    ens_values = [r['enstrophy'] for r in results]

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

    ax1.plot(speeds, ke_values, 'bo-')
    ax1.set_xlabel('Velocidad de Rotación')
    ax1.set_ylabel('Energía Cinética')
    ax1.set_title('Energía Cinética vs. Velocidad de Rotación')
    ax1.grid(True)

    ax2.plot(speeds, ens_values, 'ro-')
    ax2.set_xlabel('Velocidad de Rotación')
    ax2.set_ylabel('Enstrofía')
    ax2.set_title('Enstrofía vs. Velocidad de Rotación')
    ax2.grid(True)

    plt.tight_layout()
    plt.show()

    return simulator, results

# Función para crear una animación de la simulación
def create_animation():
    """Crea una animación de la simulación para un caso específico"""
    try:
        from matplotlib.animation import FuncAnimation

        # Crear simulador con configuración específica
        simulator = FluidSimulator(
            grid_size=30,
            sphere_radius=1.0,
            time_step=0.01,
            viscosity=0.01
        )

        # Inicializar con rotación rígida
        simulator.initialize_fluid_state(init_mode="rigid_rotation")

        # Configurar geometría toroidal
        simulator.set_container_type("toroid")

        # Establecer velocidad de rotación
        simulator.set_rotation_speed(5.0)

        # Configurar figura y eje
        fig, ax = plt.subplots(figsize=(8, 8))

        # Función de inicialización para la animación
        def init():
            ax.clear()
            return []

        # Función de actualización para cada frame
        def update(frame):
            # Ejecutar paso de simulación
            simulator.simulate_step()

            # Limpiar eje
            ax.clear()

            # Obtener corte 2D
            slice_z = simulator.grid_size // 2
            X2D, Y2D = np.meshgrid(simulator.x, simulator.y)

            # Extraer campos de velocidad en el corte
            U2D = simulator.U[:, :, slice_z]
            V2D = simulator.V[:, :, slice_z]

            # Extraer máscara para el dominio en el corte
            mask2D = simulator.mask[:, :, slice_z]

            # Aplicar máscara a velocidades
            U2D_masked = np.where(mask2D, U2D, 0)
            V2D_masked = np.where(mask2D, V2D, 0)

            # Magnitud de velocidad
            speed = np.sqrt(U2D_masked**2 + V2D_masked**2)

            # Mostrar contorno del dominio
            ax.contour(X2D, Y2D, mask2D.astype(int), levels=[0.5], colors='k', linewidths=2)

            # Mostrar campo de velocidad
            scalar_field = np.ma.masked_where(~mask2D, speed)
            cf = ax.pcolormesh(X2D, Y2D, scalar_field, cmap='viridis', shading='auto')

            # Mostrar líneas de corriente
            ax.streamplot(X2D.T, Y2D.T, U2D_masked.T, V2D_masked.T,
                         color='white', density=1.5, linewidth=0.5, arrowsize=0.8)

            # Actualizar título
            ax.set_title(f"Simulación CFD, t={simulator.time:.3f}")
            ax.set_xlabel('X')
            ax.set_ylabel('Y')
            ax.set_aspect('equal')

            return [cf]

        # Crear animación
        print("Creando animación de la simulación...")
        anim = FuncAnimation(
            fig, update, frames=50, init_func=init,
            blit=False, interval=100
        )

        # Mostrar animación
        from IPython.display import HTML
        return HTML(anim.to_jshtml())

    except ImportError:
        print("Para crear animaciones, asegúrate de tener instalado matplotlib")


# Celda 6: Ejecutar ejemplos específicos
# Esta celda muestra cómo ejecutar diferentes ejemplos de simulación

print("Simulador CFD para Fluidos en Rotación con Geometrías Variables")
print("==============================================================")
print("Esta celda contiene ejemplos de ejecución. Descomenta la simulación que deseas ejecutar:")
print()
print("# 1. Ejecutar simulación básica:")
print("# simulator = run_basic_simulation()")
print()
print("# 2. Ejecutar simulación de transformación de geometría:")
print("# demo = run_transformation_simulation()")
print()
print("# 3. Probar diferentes velocidades de rotación:")
print("# simulator, results = run_rotation_speed_test()")
print()
print("# 4. Crear animación de la simulación:")
print("# animation = create_animation()")
print()
print("# 5. Interfaz interactiva:")
print("# setup_interactive_interface()")
print()
print("Nota: Descomenta solo UNA opción a la vez.")


# Celda 7: Configuración para guardar resultados
# Esta celda permite guardar los resultados de la simulación

def save_simulation_results(simulator, filename_prefix='cfd_simulation'):
    """Guarda resultados y visualizaciones de la simulación"""
    import os
    from datetime import datetime

    # Crear carpeta para resultados si no existe
    results_dir = 'cfd_results'
    if not os.path.exists(results_dir):
        os.makedirs(results_dir)

    # Generar un identificador único para los archivos
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    file_id = f"{filename_prefix}_{timestamp}"

    # Guardar visualización 2D
    fig_2d, _ = simulator.visualize_flow()
    fig_2d.savefig(os.path.join(results_dir, f"{file_id}_2d.png"), dpi=300)
    plt.close(fig_2d)

    # Guardar visualización 3D
    fig_3d, _ = simulator.visualize_3d()
    fig_3d.savefig(os.path.join(results_dir, f"{file_id}_3d.png"), dpi=300)
    plt.close(fig_3d)

    # Guardar datos de velocidad
    import numpy as np
    np.savez(
        os.path.join(results_dir, f"{file_id}_data.npz"),
        U=simulator.U,
        V=simulator.V,
        W=simulator.W,
        P=simulator.P,
        mask=simulator.mask,
        time=simulator.time,
        grid_size=simulator.grid_size,
        sphere_radius=simulator.sphere_radius,
        container_type=simulator.container_type,
        toroid_factor=simulator.toroid_factor,
        central_axis_rotation=simulator.central_axis_rotation
    )

    print(f"Resultados guardados en la carpeta '{results_dir}' con prefijo '{file_id}'")
    return file_id

# Ejemplo de uso:
# simulator = run_basic_simulation()
# file_id = save_simulation_results(simulator, 'toroid_rotation')

    print("

    def visualize_3d(self, threshold=0.1, streamlines=True):
        """
        Crea una visualización 3D del flujo

        Args:
            threshold: Umbral para la magnitud de velocidad
            streamlines: Mostrar líneas de corriente 3D
        """
        # Crear figura 3D
        fig = plt.figure(figsize=(12, 10))
        ax = fig.add_subplot(111, projection='3d')

        # Calcular magnitud de velocidad
        speed = np.sqrt(self.U**2 + self.V**2 + self.W**2)

        # Crear máscara para velocidades mayores que el umbral
        vis_mask = (speed > threshold) & self.mask

        # Subconjunto de puntos para mayor claridad
        stride = 3
        x_subset = self.X[::stride, ::stride, ::stride]
        y_subset = self.Y[::stride, ::stride, ::stride]
        z_subset = self.Z[::stride, ::stride, ::stride]
        u_subset = self.U[::stride, ::stride, ::stride]
        v_subset = self.V[::stride, ::stride, ::stride]
        w_subset = self.W[::stride, ::stride, ::stride]
        mask_subset = vis_mask[::stride, ::stride, ::stride]

        # Visualizar vectores de velocidad
        if np.any(mask_subset):
            ax.quiver(
                x_subset[mask_subset], y_subset[mask_subset], z_subset[mask_subset],
                u_subset[mask_subset], v_subset[mask_subset], w_subset[mask_subset],
                length=0.05, normalize=True, color='b', alpha=0.7
            )

        # Visualizar contorno del dominio
        if streamlines:
            # Puntos iniciales para las líneas de corriente
            n_streamlines = 20
            x_start = np.linspace(-0.8*self.sphere_radius, 0.8*self.sphere_radius, n_streamlines)
            y_start = np.linspace(-0.8*self.sphere_radius, 0.8*self.sphere_radius, n_streamlines)
            z_start = np.linspace(-0.8*self.sphere_radius, 0.8*self.sphere_radius, n_streamlines)

            xx, yy, zz = np.meshgrid(x_start, y_start, z_start)
            x_seed = xx.flatten()
            y_seed = yy.flatten()
            z_seed = zz.flatten()

            # Filtrar puntos iniciales para que estén dentro del dominio
            seed_inside = np.zeros_like(x_seed, dtype=bool)
            for i in range(len(x_seed)):
                # Encontrar índices de malla más cercanos
                ix = np.argmin(np.abs(self.x - x_seed[i]))
                iy = np.argmin(np.abs(self.y - y_seed[i]))
                iz = np.argmin(np.abs(self.z - z_seed[i]))

                if 0 <= ix < self.grid_size and 0 <= iy < self.grid_size and 0 <= iz < self.grid_size:
                    seed_inside[i] = self.mask[ix, iy, iz]

            # Usar solo una pequeña fracción de los puntos para mayor claridad
            seed_inside = np.logical_and(seed_inside, np.random.rand(len(seed_inside)) < 0.05)

            # Crear líneas de corriente desde puntos válidos
            if np.any(seed_inside):
                for i in range(len(x_seed)):
                    if seed_inside[i]:
                        # Integrar línea de corriente
                        xyz = np.array([x_seed[i], y_seed[i], z_seed[i]])
                        xyz_points = [xyz]

                        # Integrar hacia adelante
                        for _ in range(50):
                            # Interpolar velocidad en el punto actual
                            ix = np.argmin(np.abs(self.x - xyz[0]))
                            iy = np.argmin(np.abs(self.y - xyz[1]))
                            iz = np.argmin(np.abs(self.z - xyz[2]))

                            if 0 <= ix < self.grid_size-1 and 0 <= iy < self.grid_size-1 and 0 <= iz < self.grid_size-1:
                                if self.mask[ix, iy, iz]:
                                    # Obtener velocidad en el punto
                                    vel = np.array([self.U[ix, iy, iz], self.V[ix, iy, iz], self.W[ix, iy, iz]])
                                    vel_mag = np.linalg.norm(vel)

                                    if vel_mag > 1e-10:
                                        # Avanzar en la dirección de la velocidad
                                        step_size = 0.02 * self.sphere_radius
                                        xyz_new = xyz + step_size * vel / vel_mag

                                        # Verificar si el nuevo punto está dentro del dominio
                                        ix_new = np.argmin(np.abs(self.x - xyz_new[0]))
                                        iy_new = np.argmin(np.abs(self.y - xyz_new[1]))
                                        iz_new = np.argmin(np.abs(self.z - xyz_new[2]))

                                        if 0 <= ix_new < self.grid_size and 0 <= iy_new < self.grid_size and 0 <= iz_new < self.grid_size:
                                            if self.mask[ix_new, iy_new, iz_new]:
                                                xyz = xyz_new
                                                xyz_points.append(xyz)
                                            else:
                                                break
                                        else:
                                            break
                                    else:
                                        break
                                else:
                                    break
                            else:
                                break

                        # Dibujar línea de corriente
                        if len(xyz_points) > 5:
                            xyz_points = np.array(xyz_points)
                            ax.plot(xyz_points[:, 0], xyz_points[:, 1], xyz_points[:, 2],
                                   'r-', linewidth=0.8, alpha=0.6)

        # Visualizar límites de la esfera externa
        u = np.linspace(0, 2 * np.pi, 30)
        v = np.linspace(0, np.pi, 30)
        x_sphere = self.sphere_radius * np.outer(np.cos(u), np.sin(v))
        y_sphere = self.sphere_radius * np.outer(np.sin(u), np.sin(v))
        z_sphere = self.sphere_radius * np.outer(np.ones(np.size(u)), np.cos(v))
        ax.plot_surface(x_sphere, y_sphere, z_sphere, color='gray', alpha=0.2)

        # Visualizar eje central
        theta = np.linspace(0, 2*np.pi, 30)
        z_line = np.linspace(-self.sphere_radius, self.sphere_radius, 50)

        for z in z_line:
            x_circle = self.central_axis_radius * np.cos(theta)
            y_circle = self.central_axis_radius * np.sin(theta)
            z_circle = z * np.ones_like(theta)
            ax.plot(x_circle, y_circle, z_circle, 'k-', linewidth=0.5, alpha=0.5)

        # Configurar ejes y título
        ax.set_xlim([-self.sphere_radius, self.sphere_radius])
        ax.set_ylim([-self.sphere_radius, self.sphere_radius])
        ax.set_zlim([-self.sphere_radius, self.sphere_radius])
        ax.set_xlabel('X')
        ax.set_ylabel('Y')
        ax.set_zlabel('Z')
        ax.set_title(f'Visualización 3D del flujo, t={self.time:.3f}')

        return fig, ax